## PMML模型发布DEMO

以XGBoost为例，使用PMML文件进行模型发布。
首先，我们需要有一个XGBoost模型，为此，可以以Iris数据集训练一个简单的二分类模型（只用其中的两类）。
然后利用 XGBoost 训练得到模型文件。

利用XGBoost模型转PMML的工具：https://github.com/jpmml/jpmml-xgboost
可以非常容易进行转换。

In [1]:
from sklearn.datasets import load_iris

iris = load_iris()

In [11]:
mask = iris.target < 2

X = iris.data[mask,:]
y = iris.target[mask]

In [19]:
import xgboost as xgb

params = {
    'objective' : 'reg:logistic',
    'num_round' : 10,
    'max_depth' : 3
}

dtrain = xgb.DMatrix(X, label=y)
evallist = [(dtrain, 'train')]
bst = xgb.train(params, dtrain, evals=evallist)
bst.save_model('xgb.bin')

[0]	train-rmse:0.364576
[1]	train-rmse:0.27088
[2]	train-rmse:0.203626
[3]	train-rmse:0.154579
[4]	train-rmse:0.118482
[5]	train-rmse:0.091745
[6]	train-rmse:0.071832
[7]	train-rmse:0.056919
[8]	train-rmse:0.045683
[9]	train-rmse:0.037156


#### 生成特征映射文件
特征映射文件包括三列，第一列是id，第二列是特征名，第三列是特征数据类型.
参考 <https://github.com/jpmml/jpmml-xgboost/tree/master/src/test/resources/csv>。

数据类型参考：<https://github.com/jpmml/jpmml-xgboost/blob/4cc1d8e191b58dfde1c015cf3bc0e7ff3c80a2af/src/main/java/org/jpmml/xgboost/FeatureMap.java#L66>

In [26]:
f = open('fmap.txt', 'w')
for i, fn in enumerate(iris.feature_names):
    f.write('%d\t%s\t%s\n' % (i, fn, 'q')) 
f.close()

#### 转换模型
利用 jpmml-xb

In [27]:
!java -jar converter-executable-1.2-SNAPSHOT.jar --model-input xgb.bin  --fmap-input fmap.txt  --pmml-output xgb.pmml.xml